# Tutorial: Create production ML pipelines with Python SDK v2 (preview) in a Jupyter notebook


**Learning Objectives** - By the end of this two part tutorial, you should be able to use Azure Machine Learning (Azure ML) to productionize your ML project.

This means you will be able to leverage the AzureML Python SDK to:

- connect to your Azure ML workspace
- create Azure ML data assets
- create reusable Azure ML components
- create, validate and run Azure ML pipelines
- deploy the newly-trained model as an endpoint
- call the Azure ML endpoint for inferencing

**Motivations** - This tutorial is intended to introduce Azure ML to data scientists who want to scale up or publish their ML projects. By completing a familiar end-to-end project, which starts by loading the data and ends by creating and calling an online inference endpoint, the user should become familiar with the core concepts of Azure ML and their most common usage. Each step of this tutorial can be modified or performed in other ways that might have security or scalability advantages. We will cover some of those in the Part II of this tutorial, however, we suggest the reader use the provide links in each section to learn more on each topic.

**Requirements** - In order to benefit from this tutorial, you need to have:
- basic understanding of Machine Learning projects workflow
- an Azure subscription. If you don't have an Azure subscription, [create a free account](https://aka.ms/AMLFree) before you begin.
- a working Azure ML workspace. A workspace can be created via Azure Portal, Azure CLI, or Python SDK. [Read more](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python).
- a Python environmnet
- [installed Azure Machine Learning Python SDK v2](https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/setup.sh)

## Introduction

In this tutorial, you'll create an Azure ML pipeline to train a model for credit default prediction. The pipeline handles the data preparation, training and registering the trained model.  You'll then run the pipeline, deploy the model and use it.

The image below shows the pipeline as you'll see it in the AzureML portal once submitted. It's a rather simple pipeline we'll use to walk you through the AzureML SDK v2.

The two steps are first data preparation and second training. 

![Screenshot that shows the AML Pipeline](media/pipeline-overview.jpg "Overview of the pipeline")

An AzureML pipeline that runs from local components, requires several dependent files. Fo better understanding of the project structure, we produce all these dependencies in the notebook cells. By the end of this tutorial, the project structure should look like:
```
e2e-ds-experience  
    components
        data_prep
            data_prep.py 
        train
            train.py
            train.yml   
    dependencies
        conda.yml
    deploy
        sample-request.json
    media
        metrics.jpg
        pipeline-overview.jpg
        user-logs.jpg
    e2e-ml-workflow.ipynb
```
After running this notebook, you should be able to create the project direcetly in the IDE of your choice, instead.

## Set up the pipeline resources

The Azure ML framework can be used from CLI, Python SDK, or studio interface. In this example, you'll use the AzureML Python SDK v2 to create a pipeline. 

Before creating the pipeline, you'll set up the resources the pipeline will use:

* The dataset for training
* The software environment to run the pipeline
* A compute resource to where the job will run

## Connect to the workspace

Before we dive in the code, you'll need to connect to your Azure ML workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [45]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find your Subscription ID:
1. In the upper right Azure Machine Learning Studio toolbar, select your workspace name.
1. At the bottom, select **View all properties in Azure Portal**
1. Copy the value from Azure Portal into the code.

In [46]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="191c696b-4f52-4038-9b9b-27cf6086fa00",
    resource_group_name="rg-yuenche-nn",
    workspace_name="aml-yuenche-nn",
)



The result is a handler to the workspace that you'll use to manage other resources and jobs.

> [!IMPORTANT]
> Creating MLClient will not connect to the workspace. The client initialization is lazy, it will wait for the first time it needs to make a call (in the notebook below, that will happen during dataset registration).

## Register data from an external url

The data you use for training is usually in one of the locations below:

* Local machine
* Web
* Big Data Storage services (for example, Azure Blob, Azure Data Lake Storage, SQL)
 
Azure ML uses a [`Data`](https://docs.microsoft.com/azure/machine-learning/how-to-create-register-data-assets?tabs=Python-SDK) object to register a reusable definition of data, and consume data within a pipeline. In the section below, you'll consume some data from web url as one example. `Data` assets ets from other sources can be created as well.

In [47]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

web_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

credit_data = Data(
    name="creditcard_defaults",
    path=web_path,
    type=AssetTypes.URI_FILE,
    description="Dataset for credit card defaults",
    tags={"source_type": "web", "source": "UCI ML Repo"},
    version="1.0.0",
)

This code just created a `Data` asset, it is ready to be consumed as an input by the pipeline that you'll define in the next sections. In addition, you can register the data to your workspace so it becomes reusable across pipelines.

Registering the data asset will enable you to:

* reuse and share the data asset in future pipelines
* use versions to track the modification to the data asset
* use the data asset from Azure ML designer, which is Azure ML's GUI for pipeline authoring

Since this is the first time that you're making a call to the workspace, you may be asked to authenticate. Once the authentication is complete, you'll then see the dataset registration completion message.

In [48]:
credit_data = ml_client.data.create_or_update(credit_data)
print(
    f"Dataset with name {credit_data.name} was registered to workspace, the dataset version is {credit_data.version}"
)

Dataset with name creditcard_defaults was registered to workspace, the dataset version is 1.0.0


In the future, you can fetch the same dataset from the workspace using `credit_dataset = ml_client.data.get("<DATA ASSET NAME>", version='<VERSION>')`.

## Create a Compute Resource to run our pipeline

Each step of an Azure ML pipeline can use a different compute resource for running the specific job of that step. It can be single or multi-node machines with Linux or Windows OS, or a specific compute fabric like Spark.

In this section, we provision a Linux [compute cluster](https://docs.microsoft.com/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python). See the [full list on VM sizes and prices](https://azure.microsoft.com/en-ca/pricing/details/machine-learning/) .

For this tutorial we only need a basic cluster, let's pick a Standard_DS3_v2 model with 2 vCPU cores, 7 GB RAM and create an Azure ML Compute

In [49]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        # Name assigned to the compute cluster
        name="cpu-cluster",
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.begin_create_or_update(cpu_cluster)

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

You already have a cluster named cpu-cluster, we'll reuse it as is.
AMLCompute with name cpu-cluster is created, the compute size is STANDARD_DS3_V2


## Create a job environment for pipeline steps

So far, you've created a development environment on the compute instance, your development machine. You'll also need an [environment](https://docs.microsoft.com/azure/machine-learning/concept-environments) to use for each step of the pipeline. Each step can have its own environment, or you can use some common environments for multiple steps.

In this example, you'll create a conda environment for your jobs, using a conda yaml file.
First, create a directory to store the file in.

In [51]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

Now, create the file in the dependencies directory.

In [53]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0

Writing ./dependencies/conda.yml


The specification contains some usual packages, that you'll use in your pipeline (numpy, pip).


Use the *yaml* file to create and register this custom environment in your workspace:

In [54]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.0",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name aml-scikit-learn is registered to workspace, the environment version is 0.1.0


## Build the training pipeline

Now that you have all assets required to run your pipeline, it's time to build the pipeline itself, using the Azure ML Python SDK v2.

Azure ML pipelines are reusable ML workflows that usually consist of several components. The typical life of a component is:

* Write the yaml specification of the component.
* Optionally, register the component with a name and version in your workspace, to make it reusable and shareable.
* Load that component from the pipeline code.
* Implement the pipeline using this component inputs, outputs and parameters.
* Submit the pipeline.

## Create component 1: data prep (using programmatic definition)

Let's start by creating the first component. This component handles the preprocessing of the data. The preprocessing task is performed in the *data_prep.py* python file.

First create a source folder for the data_prep component:

In [55]:
import os

data_prep_src_dir = "./components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)

This script performs the simple task of splitting the data into train and test datasets. 

[MLFlow](https://mlflow.org/docs/latest/tracking.html) will be used to log the parameters and metrics during our pipeline run.

In [56]:
%%writefile {data_prep_src_dir}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)

    credit_df = pd.read_excel(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    credit_train_df, credit_test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    credit_train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    credit_test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Writing ./components/data_prep/data_prep.py


Now that you have a script that can perform the desired task, we can create an Azure ML Component from it. Azure ML support various types of components for performing ML tasks, such as running scripts, data transfer, etc.

A component can be created by calling the component instantiators, or directly writing the defining yaml file. 

You'll use the general purpose **command** that can run command line actions. This command line action can be directly calling system commands or running a script. The inputs/outputs are accessible in the command via the `${{ ... }}` notation. For the second component of this tutorial you will use `yaml` definitions.


In [57]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep_credit_defaults",
    display_name="Data preparation for training",
    description="reads a .xl input, split the input to train and test",
    inputs={
        "data": Input(type="uri_folder"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=data_prep_src_dir,
    command="""python data_prep.py \
            --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
            --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

Optionally, register the component in the workspace for future re-use. **command()** is a component builder, in order to fetch the component itself, we need to call the **.component** property from it.

In [58]:
# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component.component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Component data_prep_credit_defaults with Version 2023-04-26-16-21-12-9216581 is registered


## Create component 2: training (using yaml definition)

The second component that you'll create will consume the training and test data, train a tree based model and return the output model. You'll use Azure ML logging capabilities to record and visualize the learning progress.

You used the `command` class to create your first component. This time you'll use the yaml definition to define the second component. Each method has its own advantages. A yaml definition can actually be checked-in along the code, and would provide a readable history tracking. Also, the same yaml file can be used in the CLI for component deficnition. The programmatic method using `command` can be easier with built-in class documentation and code completion.


Create the directory for this component:

In [59]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

Create the training script in the directory:

In [62]:
%%writefile {train_src_dir}/train.py
import argparse
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import os
import pandas as pd
import mlflow


def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--model", type=str, help="path to model file")
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_df = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_df = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.model, "trained_model"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Overwriting ./components/train/train.py


As you can see in this training script, once the model is trained, the model file is saved and registered to the workspace. Now you can use the registered model in inferencing endpoints.


For the environment of this step, you'll use one of the built-in (curated) Azure ML environments. The tag `azureml`, tells the system to use look for the name in curated environments.

First, create the *yaml* file describing the component:

In [63]:
%%writefile {train_src_dir}/train.yml
# <component>
name: train_credit_defaults_model
display_name: Train Credit Defaults Model
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder
  learning_rate:
    type: number     
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >-
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --learning_rate ${{inputs.learning_rate}}
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>


Overwriting ./components/train/train.yml


Once the `yaml` file and the script are ready, you can create your component using `load_component()`. 

In [64]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source=os.path.join(train_src_dir, "train.yml"))

Now create and register the component:

In [65]:
# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Component train_credit_defaults_model with Version 2023-04-26-16-21-50-8728890 is registered


## Create the pipeline from components

Now that both your components are defined and registered, you can start implementing the pipeline.

Here, you'll use *input data*, *split ratio* and *registered model name* as input variables. Then call the components and connect them via their inputs /outputs identifiers. The outputs of each step can be accessed via the `.outputs` property.

The python functions returned by `load_component()` work as any regular python function that we'll use within a pipeline to call each step.

To code the pipeline, we use a specific `@dsl.pipeline` decorator that identifies the Azure ML pipelines. In the decorator, we can specify the pipeline description and default resources like compute and storage. Like a python function, pipelines can have inputs, you can then create multiple instances of a single pipeline with different inputs.

Here, we used *input data*, *split ratio* and *registered model name* as input variables. We then call the components and connect them via their inputs /outputs identifiers. The outputs of each step can be accessed via the `.outputs` property.

In [66]:
# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=cpu_compute_target,
    description="E2E data_perp-train pipeline",
)
def credit_defaults_pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=pipeline_job_learning_rate,  # note: using a pipeline input as parameter
        registered_model_name=pipeline_job_registered_model_name,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
    }

Now use your pipeline definition to instantiate a pipeline with your dataset, split rate of choice and the name you picked for your model.

In [67]:
registered_model_name = "credit_defaults_model"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = credit_defaults_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=credit_data.path),
    pipeline_job_test_train_ratio=0.25,
    pipeline_job_learning_rate=0.05,
    pipeline_job_registered_model_name=registered_model_name,
)

## Submit the job 

It's now time to submit the job to run in Azure ML. This time you'll use `create_or_update`  on `ml_client.jobs`.

Here you'll also pass an experiment name. An experiment is a container for all the iterations one does on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure ML studio.

Once completed, the pipeline will register a model in your workspace as a result of training.

In [68]:
import webbrowser

# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="e2e_registered_components",
)
# open the pipeline in web browser
webbrowser.open(pipeline_job.studio_url)

False

You can track the progress of your pipeline, by using the link generated in the cell above or in this notebook using the following code:
```python
    ml_client.jobs.stream(pipeline_job.name)
```

When you select on each component, you'll see more information about the results of that component. 
There are two important parts to look for at this stage:
* `Outputs+logs` > `user_logs` > `std_log.txt`
This section shows the script run sdtout.

* `Outputs+logs` > `Metric`
This section shows different logged metrics. In this example. mlflow `autologging`, has automatically logged the training metrics.

# Create local endpoint and deployment

## Create local endpoint

The goal of a local endpoint deployment is to validate and debug your code and configuration before you deploy to Azure. Local deployment has the following limitations:
* Local endpoints *do not support* traffic rules, authentication, or probe settings.
* Local endpoints support only one deployment per endpoint

In [69]:
from azure.ai.ml.entities import (
    KubernetesOnlineEndpoint,
    KubernetesOnlineDeployment,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

from azure.ai.ml.entities._deployment.resource_requirements_settings import (
    ResourceRequirementsSettings,
)
from azure.ai.ml.entities._deployment.container_resource_settings import (
    ResourceSettings,
)

# Creating a local endpoint
import datetime

local_endpoint_name = "local-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=local_endpoint_name, description="this is a sample local endpoint"
)

In [70]:
# Install docker package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install docker

In [71]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (local-04261624142780) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'local-04261624142780', 'description': 'this is a sample local endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/local-04261624142780'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fcb6e502b30>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## Create local deployment

The example contains all the files needed to deploy a model on an online endpoint. To deploy a model, you must have:

* Model files (or the name and version of a model that's already registered in your workspace). In the example, we have a scikit-learn model that does regression.
* The code that's required to score the model. In this case, we have a score.py file.
* An environment in which your model runs. As you'll see, the environment might be a Docker image with Conda dependencies, or it might be a Dockerfile.
* Settings to specify the instance type and scaling capacity.

### Key aspects of deployment 

- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_configuration` - the configuration for the source code and scoring script
    - `path`- Path to the source code directory for scoring the model
    - `scoring_script` - Relative path to the scoring file in the source code directory
- `instance_type` - The VM size to use for the deployment.
- `instance_count` - The number of instances to use for the deployment
- `resources` - The resource ask for the deployment with requests and limits.
    - `requests` - The minimum resource ask for one deployment instance to be scheduled. For all deployment, requests for CPU and Memory should always be given.
    - `limits` - (Optional) The maximum resource that one deployment instance can use. When values in limits missing, it will take the settings in instance type. For GPU workloads, limits for CPU and Memory should also be given. If specifying GPU for your deployment, it should be in field limits, or both requests and limits with the same value.

## Gather the artifacts for model deployment 
        - Model
        - Environment
        - Score script

In [78]:
# Local deployment only supports local artifacts, therefore let's download the latest version of the model
from azure.ai.ml.operations import ModelOperations

latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

ml_client.models.download(name=registered_model_name, version=latest_model_version, download_path='./')


In [73]:
from azure.ai.ml.operations import ModelOperations

ml_client.models.download(name=registered_model_name, version=latest_model_version, download_path='./')

In [74]:
%%writefile {registered_model_name}/score.py

import os
import logging
import json
import numpy
import joblib


def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    model_path = os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "model.pkl"
    )
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    logging.info("Init complete")


def run(raw_data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info("Request received")
    data = json.loads(raw_data)["data"]
    data = numpy.array(data)
    result = model.predict(data)
    logging.info("Request processed")
    return result.tolist()

Writing credit_defaults_model/score.py


In [75]:
%%writefile {registered_model_name}/conda.yaml
channels:
- conda-forge
dependencies:
- python=3.8.5
- pip
- pip:
  - mlflow
  - cloudpickle==1.6.0
  - psutil==5.8.0
  - scikit-learn==1.0.1
  - azureml-inference-server-http
name: mlflow-env


Writing credit_defaults_model/conda.yaml


## Create local deployment

In [85]:
model = Model(path="./{model_dir}/{model_dir}/model.pkl".format(model_dir=registered_model_name))
env = Environment(
    conda_file="./{model_dir}/conda.yaml".format(model_dir=registered_model_name),
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest",
)

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./{model_dir}/".format(model_dir=registered_model_name), scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    ),
)

In [86]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Updating local deployment (local-04261624142780 / blue) 
Building Docker image from Dockerfile.
Step 1/6 : FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest
 ---> a0c1ac3e0de3
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 716d7ccd33c6
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> a67a9f88fcee
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 7378a1982bad
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 7ac24824c6b9
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> e7ed7ca472e9
Successfully built e7ed7ca472e9
Successfully tagged local-04261624142780:blue

Starting up endpoint.....Done (0m 30s)


KubernetesOnlineDeployment({'provisioning_state': 'Succeeded', 'data_collector': None, 'endpoint_name': 'local-04261624142780', 'type': 'Kubernetes', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/yuenchenn4/code/Users/yuenche/azureml-examples/tutorials/e2e-ds-experience'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fcb80433790>, 'model': Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'ee09d7c6952658978914343e8c85d34f', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/yuenchenn4/code/Users/yuenche/azureml-examples/tutorials/e2e-ds-experience'), 'creation_context': None, 'serialize': <msrest.serialization.Serial

# Verify the local deployment succeeded

## Check the status to see whether the model was deployed without error

In [87]:
ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32789/score', 'openapi_uri': None, 'name': 'local-04261624142780', 'description': 'this is a sample local endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/yuenchenn4/code/Users/yuenche/azureml-examples/tutorials/e2e-ds-experience'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fcb80558250>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## Get logs

In [88]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=local_endpoint_name, local=True, lines=100
)

'numpy==1.24.3\r\noauthlib==3.2.2\r\nopencensus==0.11.2\r\nopencensus-context==0.1.3\r\nopencensus-ext-azure==1.1.9\r\npackaging==23.1\r\npandas==2.0.1\r\nPillow==9.5.0\r\nportalocker==2.7.0\r\nprotobuf==4.22.3\r\npsutil==5.8.0\r\npyarrow==11.0.0\r\npyasn1==0.5.0\r\npyasn1-modules==0.3.0\r\npycparser==2.21\r\npydantic==1.10.7\r\nPyJWT==2.6.0\r\npyparsing==3.0.9\r\npython-dateutil==2.8.2\r\npytz==2023.3\r\nPyYAML==6.0\r\nquerystring-parser==1.2.4\r\nrequests==2.28.2\r\nrsa==4.9\r\nscikit-learn==1.0.1\r\nscipy==1.10.1\r\nsix==1.16.0\r\nsmmap==5.0.0\r\nSQLAlchemy==2.0.10\r\nsqlparse==0.4.4\r\ntabulate==0.9.0\r\nthreadpoolctl==3.1.0\r\ntyping_extensions==4.5.0\r\ntzdata==2023.3\r\nurllib3==1.26.15\r\nwebsocket-client==1.5.1\r\nWerkzeug==2.3.0\r\nwrapt==1.12.1\r\nzipp==3.15.0\r\n\r\n2023-04-26T16:40:18,564077133+00:00 | gunicorn/run | \r\n2023-04-26T16:40:18,565584455+00:00 | gunicorn/run | Entry script directory: /var/azureml-app/credit_defaults_model//.\r\n2023-04-26T16:40:18,566998275+00

## Prepare test data

In [89]:
%%writefile {registered_model_name}/sample-request.json
{
   "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
}

Overwriting credit_defaults_model/sample-request.json


## Invoke the local endpoint
Invoke the endpoint to score the model by using the convenience command invoke and passing query parameters that are stored in a JSON file

In [100]:
ml_client.online_endpoints.invoke(
    endpoint_name=local_endpoint_name,
    request_file="./{model_dir}/sample-request.json".format(model_dir=registered_model_name),
    local=True,
)

'[1, 1]'

# Deploy your online endpoint to Azure
Next, deploy your online endpoint to Azure.

## Configure online endpoint
`endpoint_name`: The name of the endpoint.

`auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` does not expire, but `aml_token` does expire. 

Optionally, you can add description, tags to your endpoint.

In [91]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "k8s-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="aks-yuenchenn",
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

## Create the endpoint

Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [92]:
ml_client.begin_create_or_update(endpoint).result()

KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://20.76.224.148/api/v1/endpoint/k8s-endpoint-04261712905175/score', 'openapi_uri': 'http://20.76.224.148/api/v1/endpoint/k8s-endpoint-04261712905175/swagger.json', 'name': 'k8s-endpoint-04261712905175', 'description': 'this is a sample online endpoint', 'tags': {'training_dataset': 'credit_defaults', 'model_type': 'sklearn.GradientBoostingClassifier'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/resourcegroups/rg-yuenche-nn/providers/microsoft.machinelearningservices/workspaces/aml-yuenche-nn/onlineendpoints/k8s-endpoint-04261712905175', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oe:f926d6f5-4428-4a65-a2af-e90086e16d79:7a4f407a-7684-475a-8c3a-6d1229a063b3?api-version=2022-02-01-preview'}, 'print_as_yaml': Tr

In [94]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "k8s-endpoint-04261712905175" with provisioning state "Succeeded" is retrieved


## Configure online deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `KubernetesOnlineDeployment` class.

In [99]:
model = Model(path="./{model_dir}/{model_dir}/model.pkl".format(model_dir=registered_model_name))
env = Environment(
    conda_file="./{model_dir}/conda.yaml".format(model_dir=registered_model_name),
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest",
)

blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./{model_dir}/".format(model_dir=registered_model_name), scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    ),
)

## Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [96]:
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint k8s-endpoint-04261712905175 exists
Uploading credit_defaults_model (0.12 MBs): 100%|██████████| 124497/124497 [00:00<00:00, 2359423.92it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.KubernetesOnlineDeployment'> and will be ignored


...........................

KubernetesOnlineDeployment({'provisioning_state': 'Succeeded', 'data_collector': None, 'endpoint_name': 'k8s-endpoint-04261712905175', 'type': 'Kubernetes', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/od:f926d6f5-4428-4a65-a2af-e90086e16d79:4ae97ec7-2df6-45bc-b8bb-c9bc41549349?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/resourceGroups/rg-yuenche-nn/providers/Microsoft.MachineLearningServices/workspaces/aml-yuenche-nn/onlineEndpoints/k8s-endpoint-04261712905175/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/yuenchenn4/code/Users/yuenche/azureml-examples/tutorials/e2e-ds-experience', 'creation_context': None, 'serialize': <msrest.serialization.S

In [97]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://20.76.224.148/api/v1/endpoint/k8s-endpoint-04261712905175/score', 'openapi_uri': 'http://20.76.224.148/api/v1/endpoint/k8s-endpoint-04261712905175/swagger.json', 'name': 'k8s-endpoint-04261712905175', 'description': 'this is a sample online endpoint', 'tags': {'training_dataset': 'credit_defaults', 'model_type': 'sklearn.GradientBoostingClassifier'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/resourcegroups/rg-yuenche-nn/providers/microsoft.machinelearningservices/workspaces/aml-yuenche-nn/onlineendpoints/k8s-endpoint-04261712905175', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/191c696b-4f52-4038-9b9b-27cf6086fa00/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oe:f926d6f5-4428-4a65-a2af-e90086e16d79:05f65fef-36c1-4ef7-a294-1d70da755b55?api-version=2022-02-01-preview'}, 'print_as_yaml': Tr

# Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [json](./model-1/sample-request.json) file. 

In [101]:
# test the blue deployment with some sample data
# comment this out as cluster under dev subscription can't be accessed from public internet.
ml_client.online_endpoints.invoke(
   endpoint_name=online_endpoint_name,
   deployment_name='blue',
   request_file="./{model_dir}/sample-request.json".format(model_dir=registered_model_name),
)

'[1, 1]'

# Managing endpoints and deployments

## Get details of the endpoint

In [102]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
http://20.76.224.148/api/v1/endpoint/k8s-endpoint-04261712905175/score


## Clean up resources

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [ ]:
# ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

## Next Steps

Learn more about [Azure ML logging](https://github.com/Azure/azureml-examples/blob/sdk-preview/notebooks/mlflow/mlflow-v1-comparison.ipynb).